# Upgrade a CellML 1.0 or CellML 1.1 model

With a new standard we often want to upgrade the models saved in the old standard to the new.
LibCellML ofers a facility to upgrade a CellML 1.0 or CellML 1.1 to CellML 2.0.
Unfortunately some models written in CellML 1.0 or CellML 1.1 that were usable with the CellML-API will not be upgradeable to CellML 2.0 and usable with libCellML without intervention.
But, for those models that require intervention libCellML will give direction on what is wrong and how to fix any errors encountered.

In this tutorial we will work through the process of upgrading the Hodgkin Huxley 1952 squid axon model.

The models for this tutorial are taken from [here](https://models.physiomeproject.org/e/e5/tutorial/tutorial.html/view).

Again we start by loading the libCellML package.

In [ ]:
import libcellml

To upgrade our model we use the *Parser* and *Importer* classes.
By default these classes are set to do their work in **strict** mode, this means they will only deal with CellML 2.0 "documents".
Because we are working with an older version of the CellML standard we need to make these classes more permissive, we do this simply by setting **strict** false.

In [ ]:
parser = libcellml.Parser()
parser.setStrict(False)

In [ ]:
importer = libcellml.Importer()
importer.setStrict(False)

Now we can load our initial model with the parser and use the importer to convert the models, the initial model imports.

To use the importer we need to do a little bit of work with the filesystem.
We import the os module.

In [ ]:
import os

Get some helpful paths ready for the importer.

In [ ]:
here = os.path.abspath(os.curdir)
base_path = os.path.join(here, "resources", "hodgkin_huxley")

As before we read in the content for the model and parse it to create the main model.

In [ ]:
with open(os.path.join("resources", "hodgkin_huxley", "stimulated.xml")) as fh:
    content = fh.read()

model = parser.parseModel(content)

Let's take a quick look at what has already happened (if anything).

In [ ]:
printer = libcellml.Printer()
print(printer.printModel(model))

Okay, now we will validate the (now) CellML 2.0 model and see if any problems have arisen.

In [ ]:
validator = libcellml.Validator()
validator.validateModel(model)

Check the validation and report on either the successful validation or the errors encountered.

In [ ]:
if validator.errorCount() == 0:
    print(f"{model.name()} is Valid.")
else:
    for i in range(validator.errorCount()):
        print(validator.error(i).description())

Now we make use of the importer.
The importer is a strange beast, it will try and read models using a relative path from the local disk to ascertain if the model is fully defined or not.
The *resolveImports* method will return whether all the imported objects were found, we can use this to report if we were able to find all objects required by the main model.

In [ ]:
if importer.resolveImports(model, base_path):
    print("We resolved all the imports in this model.")
else:
    print("Some imports in the model were not resolved.")

The by-product of this is that any CellML 1.0 or CellML 1.1 models encountered will be automatically upgraded to CellML 2.0.
We will then be able to go through all the models in the importer and determine their validity.
As we go through the importer library of models we print a simple report on the result of the validation.

In [ ]:
for i in range(importer.libraryCount()):
    k = importer.key(i)
    m = importer.library(i)
    validator.validateModel(m)
    if validator.errorCount() == 0:
        print(f"{m.name()} is Valid.")
    else:
        for i in range(validator.errorCount()):
            print(validator.error(i).description())